In [1]:
import pandas as pd
import numpy as np
import os
#from datetime import time,datetime #dtype
import time
import sys
import datetime #module

In [2]:
!{sys.executable} -m pip install businesstimedelta

In [3]:
import pytz
import businesstimedelta

In [4]:
file = pd.read_csv('Reqs_Aug_Sept.txt')

In [5]:
#Replace carriage returns with a single space
file['Action'] = file['Action'].replace('\s+', ' ', regex=True)

In [6]:
#Isolate requisitions both completed ("Requisition approved") and went to SCM("Central Buyer" and "Form Approval")
Part1 = file.loc[(file['Action'] == "Requisition approved")|(file['Action'] == "Requisition approved (via email)")]
Part2 = Part1.loc[(file['Step(s)'] == "Central Buyer") | (file['Step(s)'] == "Form Approval")]

In [7]:
#Get unique Requisition ID list and it's associated buyer
IDlist = Part2[['PR ID','User']]
IDlist.shape

(408, 2)

In [8]:
#Isolate completion times
end = file.loc[(file['Action'] == "Requisition approval process completed")]
Endtime = end[['PR ID','Date/Time']]
Endtime = Endtime.rename(columns={"Date/Time": "End Time"})
Endtime.shape

(890, 2)

In [9]:
#Merge completion times with the unique ID list
endtimeMerge = pd.merge(IDlist, Endtime, how='inner', on='PR ID', left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_actual', '_max'), copy=True, indicator=False,
         validate=None)
endtimeMerge.shape

(408, 3)

In [10]:
#Find the timestamp of Department approval
Part4 = Part1.loc[(file['Step(s)'] == "Procurement Dept Code Approval")]
Depttime = Part4[['PR ID','Date/Time']]
Depttime = Depttime.rename(columns={"Date/Time": "DeptCode Approval Time"})
Depttime.shape

(586, 2)

In [11]:
#Merge Department time with the unique ID list
EndDept = pd.merge(endtimeMerge, Depttime, how='left', on='PR ID', left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_actual', '_max'), copy=True, indicator=False,
         validate=None)
#EndDept['PR ID'].value_counts()

In [12]:
#ISSUE: Some requisitions have multple Department timestamps due to the requisitions being returned by the buyer
#Solution: Use older timestamps, which are listed second (keep ='last')
EndDept2 = EndDept.drop_duplicates(subset=['PR ID'], keep='last')

In [13]:
#Find the timestamp of Ad Hoc 1 approval
Part4 = Part1.loc[(file['Step(s)'] == "Ad Hoc 1")]
Adhoc1 = Part4[['PR ID','Date/Time']]
Adhoc1 = Adhoc1.rename(columns={"Date/Time": "Adhoc1 Approval Time"})
Adhoc1.shape

(269, 2)

In [14]:
#Merge Ad Hoc 1 with ID list
Ad1_EDtime = pd.merge(EndDept2, Adhoc1, how='left', on='PR ID', left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_actual', '_max'), copy=True, indicator=False,
         validate=None)
#Of requisitions with multiple timestamps, keep the oldest timestamp
Ad1_EDtime2 = Ad1_EDtime.drop_duplicates(subset=['PR ID'], keep='last')
Ad1_EDtime2.shape

(408, 5)

In [15]:
#Find the timestamp of Ad Hoc 2 approval
Part4 = Part1.loc[(file['Step(s)'] == "Ad Hoc 2")]
Adhoc2 = Part4[['PR ID','Date/Time']]
Adhoc2 = Adhoc2.rename(columns={"Date/Time": "Adhoc2 Approval Time"})
Adhoc2.shape

(20, 2)

In [16]:
#Merge Ad Hoc 2 with ID list
Ad2_EDtime = pd.merge(Ad1_EDtime2, Adhoc2, how='left', on='PR ID', left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_actual', '_max'), copy=True, indicator=False,
         validate=None)
Ad2_EDtime.shape

(408, 6)

In [17]:
#Find the timestamp of Associate Buyer Approval
Part4 = Part1.loc[(file['Step(s)'] == "Associate Buyer")]
AssoBuyer = Part4[['PR ID','Date/Time']]
AssoBuyer = AssoBuyer.rename(columns={"Date/Time": "Associate Buyer Approval Time"})
AssoBuyer.shape

(19, 2)

In [18]:
#Merge Associate Buyer with ID list
Asso_time = pd.merge(Ad2_EDtime, AssoBuyer, how='left', on='PR ID', left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_actual', '_max'), copy=True, indicator=False,
         validate=None)
Asso_time.shape

(408, 7)

In [19]:
#Find the timestamp of Advance Payment Approval
Part4 = Part1.loc[(file['Step(s)'] == "Advance Payment")]
AdvPay = Part4[['PR ID','Date/Time']]
AdvPay = AdvPay.rename(columns={"Date/Time": "Advance Payment Approval Time"})
AdvPay.shape

(2, 2)

In [20]:
#Merge Advance Payment to ID List
uptoAdvtimes = pd.merge(Asso_time, AdvPay, how='left', on='PR ID', left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_actual', '_max'), copy=True, indicator=False,
         validate=None)
uptoAdvtimes.shape

(408, 8)

In [21]:
#Find the PR Validation 1 Approval timestamp
Part3 = file.loc[(file['Step(s)'] == "Wait for PR Validation 1")]
PRV1 = Part3[['PR ID','Date/Time']]
PRV1 = PRV1.rename(columns={"Date/Time": "PR Validation1 Approval Time"})
PRV1.shape

(946, 2)

In [22]:
#Merge PR Validation 1 with ID list
uptoPRV1 = pd.merge(uptoAdvtimes, PRV1, how='left', on='PR ID', left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_actual', '_max'), copy=True, indicator=False,
         validate=None)
#Of requisitions with multiple timestamps, keep the oldest timestamp
Alltimes = uptoPRV1.drop_duplicates(subset=['PR ID'], keep='last')
Alltimes.shape

(408, 9)

In [23]:
#GOAL: Ensure subtractions between null and endtime do not produce errors
#GOAL: These subtractions will not become the time in queue
#Solution: Fill null with a very old datetime to produce absurd time in queues
Alltimes2 = Alltimes.fillna("01/01/1990 8:00 AM")
Alltimes2

,PR ID,User,End Time,DeptCode Approval Time,Adhoc1 Approval Time,Adhoc2 Approval Time,Associate Buyer Approval Time,Advance Payment Approval Time,PR Validation1 Approval Time
0,121296073,Tanya Krupsky,8/1/2019 10:28 AM,7/31/2019 10:21 AM,01/01/1990 8:00 AM,01/01/1990 8:00 AM,01/01/1990 8:00 AM,01/01/1990 8:00 AM,7/30/2019 2:53 PM
1,121298154,Tyrone Webb,8/21/2019 5:29 PM,7/30/2019 1:19 PM,01/01/1990 8:00 AM,01/01/1990 8:00 AM,01/01/1990 8:00 AM,01/01/1990 8:00 AM,7/30/2019 12:08 PM
2,121302001,Tanya Krupsky,8/5/2019 1:17 PM,7/30/2019 2:31 PM,01/01/1990 8:00 AM,01/01/1990 8:00 AM,01/01/1990 8:00 AM,01/01/1990 8:00 AM,7/30/2019 2:03 PM
3,121306742,Daisy Palma,8/15/2019 1:13 PM,7/30/2019 1:45 PM,8/6/2019 2:00 PM,01/01/1990 8:00 AM,01/01/1990 8:00 AM,01/01/1990 8:00 AM,7/30/2019 1:32 PM
4,121310843,Tanya Krupsky,8/5/2019 1:56 PM,8/1/2019 1:07 PM,01/01/1990 8:00 AM,01/01/1990 8:00 AM,01/01/1990 8:00 AM,01/01/1990 8:00 AM,8/1/2019 11:06 AM
5,121312578,Rosemarie Hughes,8/5/2019 7:17 AM,8/4/2019 10:29 PM,01/01/1990 8:00 AM,01/01/1990 8:00 AM,01/01/1990 8:00 AM,01/01/1990 8:00 AM,8/1/2019 2:06 PM
6,121317256,Jose Padilla,8/7/2019 12:58 PM,8/4/2019 7:26 PM,01/01/1990 8:00 AM,01/01/1990 8:00 AM,01/01/1990 8:00 AM,01/01/1990 8:00 AM,7/31/2019 12:44 PM
7,121317383,Christina Morache,8/7/2019 11:16 AM,7/31/2019 11:20 AM,8/6/2019 1:25 PM,8/6/2019 1:28 PM,01/01/1990 8:00 AM,01/01/1990 8:00 AM,7/31/2019 10:58 AM
8,121339221,Robert Puerzer,8/2/2019 10:48 AM,7/31/2019 1:11 PM,01/01/1990 8:00 AM,01/01/1990 8:00 AM,01/01/1990 8:00 AM,01/01/1990 8:00 AM,7/31/2019 8:50 AM
9,121343131,Tyrone Webb,8/15/2019 4:19 PM,8/2/2019 3:17 PM,01/01/1990 8:00 AM,01/01/1990 8:00 AM,01/01/1990 8:00 AM,01/01/1990 8:00 AM,8/2/2019 2:43 PM


In [24]:
#Convert the string to datetimes
Alltimes2['End Time'] = pd.to_datetime(Alltimes2['End Time'])
Alltimes2['DeptCode Approval Time'] = pd.to_datetime(Alltimes2['DeptCode Approval Time'])
Alltimes2['Adhoc1 Approval Time'] = pd.to_datetime(Alltimes2['Adhoc1 Approval Time'])
Alltimes2['Adhoc2 Approval Time'] = pd.to_datetime(Alltimes2['Adhoc2 Approval Time'])
Alltimes2['Associate Buyer Approval Time'] = pd.to_datetime(Alltimes2['Associate Buyer Approval Time'])
Alltimes2['Advance Payment Approval Time'] = pd.to_datetime(Alltimes2['Advance Payment Approval Time'])
Alltimes2['PR Validation1 Approval Time'] = pd.to_datetime(Alltimes2['PR Validation1 Approval Time'])

In [25]:
#ISSUE: For example, if an order is submitted Friday night, 
#the current difference would show that the order took 3 days if it was approved Monday morning.
#Solution: Just specifying normal working hours might give us better timing.

#Define a working day
workday = businesstimedelta.WorkDayRule(
    start_time=datetime.time(7),
    end_time=datetime.time(17),
    working_days=[0, 1, 2, 3, 4])

businesshrs=businesstimedelta.Rules([workday])

In [26]:
#ISSUE: Business timedelta produces output that cannot be used for subtraction
#Solution: While finding the difference, pull needed measurements and convert as needed

#Find the difference applying business hours and get the seconds
Alltimes2['End-Dept_sec'] = Alltimes2.apply(lambda x: businesshrs.difference(x['DeptCode Approval Time'], x['End Time']).seconds,axis=1)
Alltimes2['End-Ad1_sec'] = Alltimes2.apply(lambda x: businesshrs.difference(x['Adhoc1 Approval Time'], x['End Time']).seconds,axis=1)
Alltimes2['End-Ad2_sec'] = Alltimes2.apply(lambda x: businesshrs.difference(x['Adhoc2 Approval Time'], x['End Time']).seconds,axis=1)
Alltimes2['End-AB_sec'] = Alltimes2.apply(lambda x: businesshrs.difference(x['Associate Buyer Approval Time'], x['End Time']).seconds,axis=1)
Alltimes2['End-Adv_sec'] = Alltimes2.apply(lambda x: businesshrs.difference(x['Advance Payment Approval Time'], x['End Time']).seconds,axis=1)
Alltimes2['End-PRV_sec'] = Alltimes2.apply(lambda x: businesshrs.difference(x['PR Validation1 Approval Time'], x['End Time']).seconds,axis=1)  

In [27]:
#Find the difference applying business hours and get the hours
Alltimes2['End-Dept'] = Alltimes2.apply(lambda x: businesshrs.difference(x['DeptCode Approval Time'], x['End Time']).hours,axis=1)
Alltimes2['End-Ad1'] = Alltimes2.apply(lambda x: businesshrs.difference(x['Adhoc1 Approval Time'], x['End Time']).hours,axis=1)
Alltimes2['End-Ad2'] = Alltimes2.apply(lambda x: businesshrs.difference(x['Adhoc2 Approval Time'], x['End Time']).hours,axis=1)
Alltimes2['End-AB'] = Alltimes2.apply(lambda x: businesshrs.difference(x['Associate Buyer Approval Time'], x['End Time']).hours,axis=1)
Alltimes2['End-Adv'] = Alltimes2.apply(lambda x: businesshrs.difference(x['Advance Payment Approval Time'], x['End Time']).hours,axis=1)
Alltimes2['End-PRV'] = Alltimes2.apply(lambda x: businesshrs.difference(x['PR Validation1 Approval Time'], x['End Time']).hours,axis=1)

In [28]:
#Convert seconds to minutes
Alltimes2['End-Dept_sectomin'] = Alltimes2['End-Dept_sec']/60
Alltimes2['End-Ad1_sectomin'] = Alltimes2['End-Ad1_sec']/60
Alltimes2['End-Ad2_sectomin'] = Alltimes2['End-Ad2_sec']/60
Alltimes2['End-AB_sectomin'] = Alltimes2['End-AB_sec']/60
Alltimes2['End-Adv_sectomin'] = Alltimes2['End-Adv_sec']/60
Alltimes2['End-PRV_sectomin'] = Alltimes2['End-PRV_sec']/60

#convert hours to minutes
Alltimes2['End-Dept_hrtomin'] = Alltimes2['End-Dept']*60
Alltimes2['End-Ad1_hrtomin'] = Alltimes2['End-Ad1']*60
Alltimes2['End-Ad2_hrtomin'] = Alltimes2['End-Ad2']*60
Alltimes2['End-AB_hrtomin'] = Alltimes2['End-AB']*60
Alltimes2['End-Adv_hrtomin'] = Alltimes2['End-Adv']*60
Alltimes2['End-PRV_hrtomin'] = Alltimes2['End-PRV']*60

#Add minutes together
Alltimes2["End-Dept Mins"] = Alltimes2['End-Dept_hrtomin'] + Alltimes2['End-Dept_sectomin']
Alltimes2["End-Ad1 Mins"] = Alltimes2['End-Ad1_hrtomin'] + Alltimes2['End-Ad1_sectomin']
Alltimes2["End-Ad2 Mins"] = Alltimes2['End-Ad2_hrtomin'] + Alltimes2['End-Ad2_sectomin']
Alltimes2["End-AB Mins"] = Alltimes2['End-AB_hrtomin'] + Alltimes2['End-AB_sectomin']
Alltimes2["End-Adv Mins"] = Alltimes2['End-Adv_hrtomin'] + Alltimes2['End-Adv_sectomin']
Alltimes2["End-PRV Mins"] = Alltimes2['End-PRV_hrtomin'] + Alltimes2['End-PRV_sectomin']

In [29]:
#Don't know who approved before the SCM, but the distance between that person and the end time will be the shortest.
#Have Python choose the minimum difference. 
Alltimes2["Time In Queue"] = Alltimes2[['End-Dept Mins','End-Ad1 Mins','End-Ad2 Mins','End-AB Mins','End-Adv Mins','End-PRV Mins']].min(axis=1)
#Alltimes2["Time In Queue"]
Alltimes2.columns

Index(['PR ID', 'User', 'End Time', 'DeptCode Approval Time',
       'Adhoc1 Approval Time', 'Adhoc2 Approval Time',
       'Associate Buyer Approval Time', 'Advance Payment Approval Time',
       'PR Validation1 Approval Time', 'End-Dept_sec', 'End-Ad1_sec',
       'End-Ad2_sec', 'End-AB_sec', 'End-Adv_sec', 'End-PRV_sec', 'End-Dept',
       'End-Ad1', 'End-Ad2', 'End-AB', 'End-Adv', 'End-PRV',
       'End-Dept_sectomin', 'End-Ad1_sectomin', 'End-Ad2_sectomin',
       'End-AB_sectomin', 'End-Adv_sectomin', 'End-PRV_sectomin',
       'End-Dept_hrtomin', 'End-Ad1_hrtomin', 'End-Ad2_hrtomin',
       'End-AB_hrtomin', 'End-Adv_hrtomin', 'End-PRV_hrtomin', 'End-Dept Mins',
       'End-Ad1 Mins', 'End-Ad2 Mins', 'End-AB Mins', 'End-Adv Mins',
       'End-PRV Mins', 'Time In Queue'],
      dtype='object')

In [30]:
#Alltimes2["Minutes in Queue"] = Alltimes2["Time In Queue"].dt.total_seconds().div(60).astype(int)

In [32]:
#Create a subset without the extra adding/subtraction columns
TimeInQueue = Alltimes2[['PR ID','User','Time In Queue','End Time', 'DeptCode Approval Time', 'Adhoc1 Approval Time',
       'Adhoc2 Approval Time', 'Associate Buyer Approval Time',
       'Advance Payment Approval Time','PR Validation1 Approval Time']]
TimeInQueue.to_excel("TimeInQueue_withBusinessHrs.xlsx")